In [1]:
#import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium_move_cursor.MouseActions import move_to_element_chrome
import time
import pandas as pd         #to save CSV file
from bs4 import BeautifulSoup


In [2]:
#import warnings
import warnings
warnings.filterwarnings('ignore')

#chrome webdriver option
options = webdriver.ChromeOptions()
options.add_argument('--headless')
time.sleep(7)

#intialize webdriver, load web pages and click dropdown button
path = r'C:\Users\INVENTARIT\pictures\chromedriver.exe'
driver = webdriver.Chrome(path)

driver.get('https://www.doordash.com/food-delivery/san-diego-ca-restaurants/')

element=driver.find_element_by_class_name('StoreCard_iconContainer___4kCr3')
time.sleep(7)
element.click()
driver.implicitly_wait(20)

In [3]:
#parse with Beautifulsoup
result = driver.page_source
soup = BeautifulSoup(result, 'html.parser')
page =  list(soup.findAll('div', class_="StoreCard_root___1p3uN"))

In [4]:
#define lists
ea = []
fa = []
ga= []
ha = []
lb = []

In [5]:
#loop over stores and scrape details
for i in page:
    ea.append(i.find("h2", {"class": "StoreCard_storeTitle___1tIOi"}).get_text().strip()) #name
    fa.append(i.find("p", {"StoreCard_storeInfo___3EpLG"}).get_text().strip()) #store info
    if i.find("p", {"StoreCard_storeReviews___8EiRe"}) is None: #rating
              ga.append('None')
    else:
              ga.append(i.find("p", {"StoreCard_storeReviews___8EiRe"}).get_text().strip())
    ha.append(i.findAll("h3", {"StoreCard_itemTitle___2d6RQ"})) #menu
   

    lb.append(i.findAll("div", {"StoreCard_itemPrice___1_VKq"})) #price

In [6]:
df = pd.DataFrame({'Restaurant':ea, 'Location_and_type': fa, 'Rating': ga}) #ist dataframe

In [7]:
#create data frame for nested list of Menu and price
gf = pd.DataFrame(ha)
ef = pd.DataFrame(lb)

In [8]:
#function to get text of Menu and price
def my_fun(x, var1):
    if (x[var1]) is None :
       x[var1] = 'None'
    else:
        x[var1]= x[var1].get_text()
    return x    

In [9]:
#apply function to Menu df
gf.apply(lambda x: my_fun(x,0), axis=1)
gf.apply(lambda x: my_fun(x,1), axis=1)
gf.apply(lambda x: my_fun(x,2), axis=1)
gf['Menu'] = gf[gf.columns[0:]].apply(
lambda x: ','.join(x.dropna().astype(str)),axis=1)
gf = gf.drop([0,1,2], axis = 1)


In [10]:
#merge Menu df with first df containing Restaurant, location and rating
Newdf = df.merge(gf, left_index=True, right_index=True)

In [11]:
#apply function to price df
ef.apply(lambda x: my_fun(x,0), axis=1)
ef.apply(lambda x: my_fun(x,1), axis=1)
ef.apply(lambda x: my_fun(x,2), axis=1)
ef['Price'] = ef[ef.columns[0:]].apply(
lambda x: ','.join(x.dropna().astype(str)),axis=1)
ef = ef.drop([0,1,2], axis = 1)


In [12]:
#merge price df to Newdf for the final dataframe
finaldf = Newdf.merge(ef, left_index=True, right_index=True)

In [13]:
#examine final df
finaldf.head()

,Restaurant,Location_and_type,Rating,Menu,Price
0,DashMart,San Diego • Convenience Store • $,543 ratings,"None,None,None","None,None,None"
1,The Melt,San Diego • American • $,"4.81,675 ratings","BBQ Bacon Meltburger,BBQ Bacon Chicken,Impossi...","$12.49,$12.49,$17.49"
2,Alfredo’s Pizzeria & Restaurant,San Diego • Italian • $$,4.829 ratings,"Mozzarella Sticks (2Pcs),French Fries,None","$9.99,$4.99,None"
3,The Cheesecake Factory,San Diego • Dessert • $$,"4.85,571+ ratings","Pineapple Moscow Mule,Ultimate Margarita,Passi...","$24.00,$24.00,$24.00"
4,Colima's Mexican Food,San Diego • Latin American • $,"4.75,883+ ratings","None,None,None","None,None,None"


In [14]:
#export file to csv
finaldf.to_csv('Doordash San Diego.csv', encoding = 'utf-8', sep=',')